In [6]:
# Making imports convenient
import sys
import os
PATH=os.getcwd().split('/notebooks')[0]
sys.path.insert(1, PATH)

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset,concatenate_datasets
import transformers
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

from transformers import AutoTokenizer, DataCollatorWithPadding,AutoModelForSequenceClassification,AdamW,get_scheduler,TrainingArguments,Trainer,EarlyStoppingCallback
from sklearn.model_selection import ParameterGrid
from src.utils.myutils import *
import yaml
import json

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


model_name = 'ufal/robeczech-base'
CONFIG_PATH = PATH + '/src/utils/config.yaml'
SUBJ_MODEL_PATH = '/home/horyctom/bias-detection-thesis/src/models/trained/wnc_cs_pretrained.pth'

training_args = TrainingArguments(
            output_dir = './',
            num_train_epochs=3,
            save_total_limit=2,
            disable_tqdm=False,
            per_device_train_batch_size=16,  
            warmup_steps=0,
            weight_decay=0.1,
            logging_dir='./',
            learning_rate=2e-5)

BATCH_SIZE = 16
transformers.utils.logging.set_verbosity_error()

In [7]:
import logging
logging.disable(logging.ERROR)

In [8]:
data_babe = load_dataset('csv',data_files = PATH + '/data/CS/processed/BABE/train.csv')['train']
data_cwnc = load_dataset('csv',data_files = PATH + '/data/CS/processed/CWNC/train.csv')['train']
data_wnc = load_dataset('csv',data_files = PATH + '/data/CS/processed/WNC/wnc.csv')['train']

0 tables [00:00, ? tables/s]

ParserError: Error tokenizing data. C error: Expected 2 fields in line 41257, saw 3


In [ ]:


with open(CONFIG_PATH) as f:
    config_data = yaml.load(f, Loader=yaml.FullLoader)
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False,padding=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

babe_tok = preprocess_data(data_babe,tokenizer,'sentence')
cwnc_tok = preprocess_data(data_cwnc,tokenizer,'sentence')
wnc_tok = preprocess_data(data_wnc,tokenizer,'sentence')

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/horyctom/.cache/huggingface/datasets/csv/default-0dac36967151feb1/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


  0%|          | 0/363 [00:00<?, ?ba/s]

In [4]:
for idx,s in enumerate(data_wnc['sentence']):
    if s is None:
        print(idx)

41255
94387
158611
182006
198892
262959
280454
327546
